In [2]:
from statsbombpy import sb
import pandas as pd

In [3]:
# Get 360 frames
frames = sb.competition_frames(
    country="Europe",
    division= "UEFA Euro",
    season="2020"
)

/home/agale/.local/lib/python3.8/site-packages/statsbombpy/api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)


In [9]:
# Read likelihoods from xp.csv
xp = pd.read_csv('../web/data/xp.csv')
xp

,id,likelihood
0,94dbc5c3-ef37-445e-9154-3d9f9ea9245d,0.996895
1,c943a37b-f9ae-4784-beb7-f9efce2ca855,0.438872
2,2bf9d325-fba4-4dfa-9f9f-08a25e70b073,0.825905
3,842fd78c-c050-4a88-b48d-758414697864,0.953306
4,60034587-15ac-4edd-bf3d-19a059043557,0.661221
...,...,...
54815,3dca66b8-ba56-40c3-b64e-9264644c21ff,0.984714
54816,3b3da85b-158e-4096-bd7f-154fad5cfb7e,0.974524
54817,e0899d5f-e66c-452a-b7d7-f261b746efd8,0.356846
54818,b57da8ed-e435-4375-9291-ce1ce9fe61bb,0.929796


In [13]:
match_ids = frames['match_id'].unique()
match_ids

array([3788753, 3788765, 3795107, 3795221, 3795506, 3794688, 3794692,
       3794686, 3794687, 3794685, 3788774, 3788771, 3788772, 3788768,
       3788767, 3788763, 3788759, 3788758, 3788757, 3788745, 3788746,
       3794691, 3794690, 3794689, 3788769, 3788761, 3788764, 3788766,
       3788754, 3788755, 3788773, 3788762, 3788760, 3788747, 3788741,
       3795220, 3788770, 3795108, 3795187, 3795109, 3788756, 3788749,
       3788750, 3788748, 3788751, 3788752, 3788742, 3788775, 3788776,
       3788743, 3788744])

In [14]:
for match_id in match_ids:
    flat_events = sb.events(match_id=match_id, split=True, flatten_attrs=False)
    passes = pd.DataFrame(flat_events['passes'])
    passes = passes.join(xp.set_index('id'), on='id', how='left')
    passes['outcome'] = [p['outcome']['name'] if 'outcome' in p else 'Complete' for p in passes['pass']]
    passes['recipient_id'] = [p['recipient']['id'] if 'recipient' in p else None for p in passes['pass']]
    passes['recipient_name'] = [p['recipient']['name'] if 'recipient' in p else None for p in passes['pass']]
    passes['location_end'] = [p['end_location'] if 'end_location' in p else None for p in passes['pass']]
    passes['height'] = [p['height']['name'] if 'height' in p else None for p in passes['pass']]
    passes['type'] = [p['type']['name'] if 'type' in p else None for p in passes['pass']]
    passes['body_part'] = [p['body_part']['name'] if 'body_part' in p else None for p in passes['pass']]
    passes['location_x'] = [p[0] for p in passes['location']]
    passes['location_y'] = [p[1] for p in passes['location']]
    passes['location_end_x'] = [p[0] for p in passes['location_end']]
    passes['location_end_y'] = [p[1] for p in passes['location_end']]
    passes = passes.drop(columns=[
        'related_events',
        'pass',
        'location',
        'location_end',
        'timestamp',
        'second',
        'possession_team_id',
        'possession_team'
    ])
    passes = passes.astype({'recipient_id': 'Int64'})
    match_frames = frames[frames["match_id"] == match_id]
    match_frames = match_frames.drop(columns=["visible_area"])
    for (_, frame) in match_frames.iterrows():
        uuid = frame['event_uuid']
        freeze = frame['freeze_frame']
        actor = next((x for x in freeze if x['actor']), None)
        if actor is not None:
            location = actor['location']
            row = passes.index[passes['id'] == uuid]
            if len(row) > 0:
                passes.iat[row[0], passes.columns.get_loc('location_x')] = location[0]
                passes.iat[row[0], passes.columns.get_loc('location_y')] = location[1]
    match_frames.to_csv('../web/data/frames/' + str(match_id) + '.csv', index=False)
    passes.to_csv('../web/data/passes/' + str(match_id) + '.csv', index=False)

/home/agale/.local/lib/python3.8/site-packages/statsbombpy/api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)
/home/agale/.local/lib/python3.8/site-packages/statsbombpy/api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)
/home/agale/.local/lib/python3.8/site-packages/statsbombpy/api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)
/home/agale/.local/lib/python3.8/site-packages/statsbombpy/api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)
/home/agale/.local/lib/python3.8/site-packages/statsbombpy/api_client.py

/home/agale/.local/lib/python3.8/site-packages/statsbombpy/api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)
/home/agale/.local/lib/python3.8/site-packages/statsbombpy/api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)
/home/agale/.local/lib/python3.8/site-packages/statsbombpy/api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)
/home/agale/.local/lib/python3.8/site-packages/statsbombpy/api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)
/home/agale/.local/lib/python3.8/site-packages/statsbombpy/api_client.py